In [96]:
#% pip install openpyxl
import os
import pandas as pd
import configparser
import sys
from __globals import *
sys.path.append("../tools/")
from _utilities import prepare_database
import _db_tools as db
####Helper Functions####
def limits_gen(means, *argv):
    '''Generate the limit values for a list containing the means. Supports
    common (one) tolerances and single (low and high) tolerances'''
    HI_LIMIT = []; LO_LIMIT = [] #Preallocation
    if len(argv) == 1: 
        l_tolerance = argv[0]; h_tolerance = argv[0]
    elif len(argv) == 2:
        l_tolerance = argv[0]; h_tolerance = argv[1]
    for mean in means:
        high_limit = mean + l_tolerance
        HI_LIMIT.append(round(high_limit, 4))
        low_limit = mean - h_tolerance
        LO_LIMIT.append(round(low_limit, 4))
    limits = pd.DataFrame([LO_LIMIT, HI_LIMIT]).transpose()
    limits.columns = ["LO_LIMIT", "HI_LIMIT"]
    return limits

def ini_generator_personalized(LIMITS):
    '''Generates a ini file with personalized limits for every mean'''
    class CaseSensitiveConfigParser(configparser.ConfigParser):
        '''A custom class to override optionxform and avoid uppercases being converted to lowercase
        It just works F76 F76 F76 F76 F76'''
        def optionxform(self, optionstr):
            return optionstr
    config = CaseSensitiveConfigParser()
    config.read('../data/template.ini') #Import a template
    keys_list = []
    for section_name in config.sections(): #Get a keys list with the correct uppercased keys
        section = config[section_name]
        keys_list.extend(section.keys())
    HI_LIMIT = LIMITS.iloc[:, 1]
    LO_LIMIT = LIMITS.iloc[:, 0]
    for section in config.sections(): #Iterate through the sections and options in the .ini file
        keys_list = list(config[section].keys())
        j = 0
        for i in range(0, len(keys_list), 2):
            key1 = keys_list[i]
            key2 = keys_list[i + 1]
            col1 = str(LIMITS.iloc[j, 1])
            col2 = str(LIMITS.iloc[j, 0])
            j += 1
            config[section][key1] = col1
            config[section][key2] = col2
    for section in config.sections(): #Print the five first elements of the .ini for a quick check
        print(f"[{section}]")
        i = 0
        for key, value in config.items(section): 
            if i < 5:
                print(f"{key} = {value}")
                i += 1
            else:
                break
        print("...")
    #Save the modified data to a new .ini file
    with open(f'../a2_output/{tooling}.ini', 'w') as configfile:
        for section in config.sections():
            configfile.write(f"[{section}]\n")
            keys = keys_list #Recover the original keys to write them in the .ini file
            for i, key in enumerate(keys):
                configfile.write(f"{key} = {config[section][key]}\n")
                if (i + 1) % 4 == 0 and i < len(keys) - 1: #Insert a blank line every four keys
                    configfile.write("\n")



In [97]:
#Data preparation
dbh = db.SQLite_Data_Extractor("database.db") #Connect to the database
df = dbh.retrieve(glob.tooling) #Get the desired tooling data
dbh.close_conn() 

../database/database.db found.
Table PASSAT_B9 retrieved succesfully.
Closed connection to: ../database/database.db


In [98]:
resume = df.transpose().describe() #Transpose the df first due to describe() working in columns.
rough_means = list(resume.iloc[1, :].values)
means = []; means_fbx = []; means_fby = [] #Preallocation
for i, mean in enumerate(rough_means): #Iterates and rounds every mean value
    mean = round(mean, 4)
    means_fbx.append(mean) if i % 2 == 0 else means_fby.append(mean)
    means.append(mean)
print("Mean per fiber") 
print("All fibers: " + str(means))
print("fiber x: " + str(means_fbx))
print("fiber y: " + str(means_fby))

Mean per fiber
All fibers: [0.3312, 0.3543, 0.3354, 0.3564, 0.3418, 0.3622, 0.3324, 0.3556, 0.3267, 0.3522, 0.334, 0.3558, 0.3315, 0.3546, 0.3247, 0.3508, 0.3268, 0.3524, 0.3357, 0.3561, 0.3302, 0.3516, 0.3301, 0.3538]
fiber x: [0.3312, 0.3354, 0.3418, 0.3324, 0.3267, 0.334, 0.3315, 0.3247, 0.3268, 0.3357, 0.3302, 0.3301]
fiber y: [0.3543, 0.3564, 0.3622, 0.3556, 0.3522, 0.3558, 0.3546, 0.3508, 0.3524, 0.3561, 0.3516, 0.3538]


In [105]:
#Generates the limits and stores them in a dataframe
LIMITS = limits_gen(means, glob.tolerance)
LIMITS_X = limits_gen(means_fbx, glob.tolerance)
LIMITS_Y = limits_gen(means_fby, glob.tolerance)
#Alternative with different high and low tolerances: 
'''LIMITS = limits_gen(means, glob.lo_tolerance, glob.hi_tolerance)'''
print("Personalized limits with the given tolerance:")
LIMITS

Personalized limits with the given tolerance:


,LO_LIMIT,HI_LIMIT
0,0.3162,0.3462
1,0.3393,0.3693
2,0.3204,0.3504
3,0.3414,0.3714
4,0.3268,0.3568
5,0.3472,0.3772
6,0.3174,0.3474
7,0.3406,0.3706
8,0.3117,0.3417
9,0.3372,0.3672


In [100]:
##Creation of the .ini
confirmation = input("Do you want to create a new .ini file (y/n): ").strip().lower()
if confirmation == 'y':
    ini_generator_personalized(LIMITS)
else:
    print("Operation canceled.")

Operation canceled.


In [101]:
##Creation of the target dataframe with correct limits
new_column_names = [f'{i+1}' for i in range(len(df.columns))] #Columns renaming
df.columns = new_column_names
new_row_names = []; counter = 1; fiber = "X" #Preallocation
for i in range(0, len(df)): #Rows renaming
    new_row_name = f'Guia_Luz_Blanco_FB{counter}_{fiber}' 
    new_row_names.append(new_row_name)
    if i % 2 == 1: #Alternate the fiber axis and update the fober number every two fibers
        counter += 1
        fiber = "X"
    else:
        fiber = "Y"
df.index = new_row_names
LIMITS.index = new_row_names
target = pd.concat([df, LIMITS], axis=1) #Concatenates the measures and limits
target


,1,2,3,4,5,6,7,8,9,10,...,23,24,25,26,27,28,29,30,LO_LIMIT,HI_LIMIT
Guia_Luz_Blanco_FB1_X,0.3316,0.3316,0.3311,0.3314,0.3308,0.3298,0.3316,0.3316,0.3302,0.3298,...,0.3316,0.3316,0.3316,0.3315,0.3316,0.3316,0.3316,0.3316,0.3162,0.3462
Guia_Luz_Blanco_FB1_Y,0.3546,0.3546,0.3543,0.3544,0.3540,0.3534,0.3546,0.3546,0.3536,0.3534,...,0.3546,0.3546,0.3546,0.3545,0.3546,0.3546,0.3546,0.3546,0.3393,0.3693
Guia_Luz_Blanco_FB2_X,0.3355,0.3355,0.3357,0.3356,0.3357,0.3350,0.3357,0.3356,0.3356,0.3352,...,0.3357,0.3353,0.3353,0.3356,0.3346,0.3349,0.3357,0.3351,0.3204,0.3504
Guia_Luz_Blanco_FB2_Y,0.3565,0.3565,0.3564,0.3565,0.3565,0.3562,0.3564,0.3565,0.3564,0.3563,...,0.3564,0.3563,0.3564,0.3565,0.3562,0.3562,0.3565,0.3565,0.3414,0.3714
Guia_Luz_Blanco_FB3_X,0.3423,0.3424,0.3414,0.3414,0.3414,0.3416,0.3412,0.3421,0.3414,0.3412,...,0.3423,0.3420,0.3423,0.3422,0.3423,0.3423,0.3411,0.3411,0.3268,0.3568
Guia_Luz_Blanco_FB3_Y,0.3624,0.3625,0.3623,0.3623,0.3622,0.3623,0.3619,0.3623,0.3623,0.3619,...,0.3625,0.3622,0.3623,0.3623,0.3624,0.3624,0.3618,0.3618,0.3472,0.3772
Guia_Luz_Blanco_FB4_X,0.3325,0.3325,0.3321,0.3325,0.3325,0.3322,0.3323,0.3325,0.3324,0.3318,...,0.3325,0.3325,0.3325,0.3325,0.3325,0.3325,0.3323,0.3320,0.3174,0.3474
Guia_Luz_Blanco_FB4_Y,0.3557,0.3557,0.3555,0.3557,0.3557,0.3556,0.3556,0.3557,0.3557,0.3554,...,0.3557,0.3557,0.3557,0.3557,0.3557,0.3557,0.3556,0.3554,0.3406,0.3706
Guia_Luz_Blanco_FB5_X,0.3267,0.3267,0.3267,0.3267,0.3267,0.3267,0.3267,0.3267,0.3267,0.3267,...,0.3267,0.3267,0.3267,0.3267,0.3267,0.3267,0.3267,0.3267,0.3117,0.3417
Guia_Luz_Blanco_FB5_Y,0.3522,0.3522,0.3522,0.3522,0.3522,0.3522,0.3522,0.3522,0.3522,0.3522,...,0.3522,0.3522,0.3522,0.3522,0.3522,0.3522,0.3522,0.3522,0.3372,0.3672


In [102]:
##Data export
confirmation = input("Do you want to export the data to a new Target.xlsx file? (y/n): ").strip().lower()
if confirmation == 'y':
    target.to_excel('../a2_output/Target_with_limits.xlsx', index=True, header=True)
    #os.startfile('../a2_output/Target_with_limits.xlsx')
    print('Data exported in: ../a2_output/Target_with_limits.xlsx')
else:
    print("Operation canceled.")

Operation canceled.


In [103]:
##Update of the database table
confirmation = input("Do you want to export the new limits to your database (y/n): ").strip().lower()
if confirmation == 'y':
    prepare_database(LIMITS, tooling+"_limits") #Store a df inside the database of the project
else:
    print("Operation canceled.")

Operation canceled.
